In [17]:
import numpy as np
import pandas as pd
import os
from sklearn.decomposition import PCA
import tqdm

## Submission template

In [2]:
## load template
template_df = pd.read_csv('../data/sample_submission.csv')
assert template_df.shape[0] == 255
template_df.head()

,id,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
gene_list = template_df.columns[1:].tolist()

In [4]:
len(gene_list)

18211

In [5]:
gene2idx = dict()
idx2gene = dict()

for idx, gene in enumerate(gene_list):
    gene2idx[gene] = idx
    idx2gene[idx] = gene

del idx, gene

## Multiome data

In [6]:
df = pd.read_parquet('../data/multiome_train.parquet')
print(df.shape)
df.head()

(216251368, 4)


,obs_id,location,count,normalized_count
0,000225c1151ab841,AAMP,1,6.320659
1,000225c1151ab841,AASS,1,6.320659
2,000225c1151ab841,ABCC11,1,6.320659
3,000225c1151ab841,ABCC2,1,6.320659
4,000225c1151ab841,ABR,1,6.320659


In [7]:
df = df.drop(columns=['count'])
df.head()

,obs_id,location,normalized_count
0,000225c1151ab841,AAMP,6.320659
1,000225c1151ab841,AASS,6.320659
2,000225c1151ab841,ABCC11,6.320659
3,000225c1151ab841,ABCC2,6.320659
4,000225c1151ab841,ABR,6.320659


In [8]:
df = df[df['location'].isin(gene_list)]
df.head()

,obs_id,location,normalized_count
0,000225c1151ab841,AAMP,6.320659
1,000225c1151ab841,AASS,6.320659
3,000225c1151ab841,ABCC2,6.320659
4,000225c1151ab841,ABR,6.320659
5,000225c1151ab841,ABRAXAS2,6.320659


In [9]:
df.shape

(40845122, 3)

In [10]:
obsid2idx = dict()
idx2obsid = dict()

for idx, obsid in enumerate(df['obs_id'].unique().tolist()):
    obsid2idx[obsid] = idx
    idx2obsid[idx] = obsid

del idx, obsid

In [11]:
len(obsid2idx)

25551

In [12]:
len(idx2obsid)

25551

## Look up cell types

In [27]:
meta = pd.read_csv('../data/multiome_obs_meta.csv')
meta = meta.drop(columns=['donor_id'])
print(meta.shape)
meta.head()

(25551, 2)


,obs_id,cell_type
0,000225c1151ab841,B cells
1,0003c40a54367871,T cells CD4+
2,0004bf574b822c3c,T cells CD4+
3,000d59b5478f28e2,B cells
4,0011b7473923d7b5,NK cells


In [28]:
meta['obs_id'].nunique()

25551

In [29]:
obsid2celltype = dict()

for i in range(meta.shape[0]):
    obs_id = meta.iloc[i]['obs_id']
    cell_type = meta.iloc[i]['cell_type']
    obsid2celltype[obs_id] = cell_type

In [30]:
meta['cell_type'].value_counts()

cell_type
T cells CD4+          9371
Myeloid cells         6874
NK cells              4497
B cells               3809
T cells CD8+           508
T regulatory cells     492
Name: count, dtype: int64

In [31]:
len(np.setdiff1d(df['obs_id'].unique(), meta['obs_id'].unique()))

0

In [32]:
len(np.setdiff1d(meta['obs_id'].unique(), df['obs_id'].unique()))

0

In [36]:
celltype2cnt = meta['cell_type'].value_counts().to_dict()
celltype2cnt

{'T cells CD4+': 9371,
 'Myeloid cells': 6874,
 'NK cells': 4497,
 'B cells': 3809,
 'T cells CD8+': 508,
 'T regulatory cells': 492}

In [34]:
type(meta['cell_type'].value_counts().to_dict()['B cells'])

int

In [35]:
celltype_list = meta['cell_type'].unique().tolist()
celltype_list

['B cells',
 'T cells CD4+',
 'NK cells',
 'Myeloid cells',
 'T regulatory cells',
 'T cells CD8+']

## PCA

In [16]:
M = np.zeros((len(obsid2idx), len(gene2idx)))

for i in tqdm.tqdm(range(df.shape[0])):
    obsid_idx = obsid2idx[df.iloc[i]['obs_id']]
    gene_idx = gene2idx[df.iloc[i]['location']]
    val = df.iloc[i]['normalized_count'].astype(float)

    M[obsid_idx, gene_idx] = val

100%|████████████████████████████████████████████████████████| 40845122/40845122 [41:06<00:00, 16556.84it/s]


In [51]:
def pca_reduction(M, n_components):
    
    # PCA
    pca = PCA(n_components=n_components)
    pca.fit(M)
    
    #print(pca.explained_variance_ratio_)
    print(sum(pca.explained_variance_ratio_))
    
    M_red = pca.transform(M)
    print(M_red.shape)
    
    # categorized by cell types
    celltype2geneexp = dict()

    for cell_type in celltype_list:
        celltype2geneexp[cell_type] = np.array([0 for _ in range(n_components)])

    for i in range(M_red.shape[0]):
        
        obs_id = idx2obsid[i]
        cell_type = obsid2celltype[obs_id]

        if obs_id not in obsid2celltype:
            continue

        celltype2geneexp[cell_type] = celltype2geneexp[cell_type] + M_red[i, :]/(1.0*celltype2cnt[cell_type])
    
    # convert to dataframe
    cell_emb_df = pd.DataFrame(columns=['cell_type']+['PC_'+str(i) for i in range(1, n_components+1)])
    
    for k, v in celltype2geneexp.items():
        cell_emb_df.loc[len(cell_emb_df.index)] = [k] + v.tolist()
        
    file_name = './cell_emb_normcnt_pca_' + str(n_components) + '.csv'
    
    cell_emb_df.to_csv(file_name, index=False)
    
    tmp = pd.read_csv(file_name)
    print(tmp.shape)
    del tmp
    
    return cell_emb_df

In [52]:
pca_reduction(M=M, n_components=16)

0.10622993902630974
(25551, 16)
(6, 17)


,cell_type,PC_1,PC_2,PC_3,PC_4,PC_5,PC_6,PC_7,PC_8,PC_9,PC_10,PC_11,PC_12,PC_13,PC_14,PC_15,PC_16
0,B cells,-16.645466,-9.937811,38.389760,-13.680453,3.392247,0.573510,1.166997,-1.193820,-1.034064,-0.292658,0.150563,0.050403,0.010908,0.262502,-0.372228,0.059870
1,T cells CD4+,-20.301091,21.494592,-4.919826,3.948149,-8.514178,0.414083,-0.534505,-1.421001,-0.040458,0.055026,-0.595126,-0.026152,0.076710,0.269101,0.294578,0.652199
2,NK cells,-24.776161,2.033651,-15.259671,-3.799439,20.975446,-0.586868,1.444727,2.714108,-0.113113,-0.119557,0.521539,0.435839,-0.624680,-0.348762,0.002319,-1.325414
3,Myeloid cells,57.544910,-26.085601,-4.039023,3.930486,-2.621671,-0.661969,-1.015971,0.582177,0.092216,0.145742,0.120291,0.164485,-0.014980,-0.377656,-0.434599,0.094791
4,T regulatory cells,-31.188498,0.774462,-4.064170,6.189435,-5.682527,1.086418,-2.508905,-4.408511,2.837978,-0.008703,3.448481,-3.421481,11.269329,3.688374,7.130860,0.597393
5,T cells CD8+,-29.835574,12.231034,-3.418006,4.199702,-13.078978,1.161660,4.498008,7.529962,5.504666,0.273984,0.264842,-2.665720,-6.678634,-2.306904,-3.689088,-2.608125


In [53]:
pca_reduction(M=M, n_components=1000)

0.3572867315115667
(25551, 1000)
(6, 1001)


,cell_type,PC_1,PC_2,PC_3,PC_4,PC_5,PC_6,PC_7,PC_8,PC_9,...,PC_991,PC_992,PC_993,PC_994,PC_995,PC_996,PC_997,PC_998,PC_999,PC_1000
0,B cells,-16.645466,-9.937811,38.389760,-13.680453,3.392247,0.573510,1.166996,-1.193825,-1.034051,...,0.032999,-0.003236,0.013568,0.032793,-0.003595,-0.000152,0.022934,-0.000098,-0.012523,-0.001455
1,T cells CD4+,-20.301091,21.494592,-4.919826,3.948149,-8.514178,0.414082,-0.534504,-1.421013,-0.040454,...,-0.024949,0.022010,-0.005641,-0.019585,0.020725,-0.016510,-0.015397,-0.012385,0.012747,-0.005554
2,NK cells,-24.776161,2.033651,-15.259671,-3.799439,20.975446,-0.586868,1.444725,2.714108,-0.113097,...,-0.008559,-0.023819,-0.029611,0.026605,-0.017015,0.016960,0.015704,-0.005299,0.006463,0.025425
3,Myeloid cells,57.544910,-26.085601,-4.039023,3.930486,-2.621671,-0.661970,-1.015971,0.582191,0.092228,...,0.014131,0.002084,0.009278,-0.013473,-0.012390,0.001317,-0.004089,0.013217,-0.004310,-0.005791
4,T regulatory cells,-31.188498,0.774462,-4.064170,6.189435,-5.682527,1.086446,-2.508867,-4.408419,2.837998,...,-0.037547,0.039340,0.023422,0.056644,-0.097037,-0.026429,-0.057392,0.104181,-0.140718,-0.102839
5,T cells CD8+,-29.835574,12.231034,-3.418006,4.199702,-13.078978,1.161673,4.497970,7.529941,5.504173,...,0.133715,-0.237190,0.116231,0.007329,0.056895,0.163347,0.083964,-0.003633,-0.003846,0.066247
